In [ ]:
""" IN THE NAME OF  ALLAH , THE MOST GRACIOUS, THE MOST MERCIFUL. """

In [1]:
import sys
import cv2
import time
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F
import torchvision.transforms as T
from torch import Tensor
from torch.utils.data import DataLoader, random_split

modules_path = '../external_modules/'
dataset_path = '../../Dataset/STMM'
save_path = "../model"

sys.path.insert(1, modules_path)
from video_dataset import VideoFrameDataset, ImglistToTensor
from global_defines import GESTURE_MAP_NUM2STR

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

C:\Users\PMLS\anaconda3\Lib\site-packages\torchvision\io\image.py:14: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\PMLS\anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


cpu


In [2]:
# Load dataset
#img_w = 640
#img_h = 480
#frames_per_video = 7
batch_size = 8
num_workers = 1

dataset = VideoFrameDataset(
    root_path= f"{dataset_path}",
    annotationfile_path=f"{dataset_path}/annotations.txt",
    num_segments=8,
    frames_per_segment=1,
    imagefile_template='{:01d}.jpg',
    transform=ImglistToTensor(),
    test_mode=False
)
class_map = {0:"down", 1:"left", 2:"right", 3:"up"}

train_dataset, valid_dataset, test_dataset = random_split(dataset, [0.7,0.2,0.1])

train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, num_workers=num_workers, shuffle=True)

In [3]:
# Define preprocessings
import mediapipe as mp

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.35,
    min_tracking_confidence=0.35
)
mp_drawing = mp.solutions.drawing_utils

def get_hand_landmarks_frame(frame, transform=False):
    if transform==True:
        frame = 255*frame
        frame = frame.transpose(2,0).numpy().astype('uint8')
    results = hands.process(frame)
    
    hand_landmarks = []
    if results.multi_hand_landmarks:
        for hand_landmark in results.multi_hand_landmarks[0].landmark:
            hand_landmarks.append([hand_landmark.x, hand_landmark.y, hand_landmark.z])
        hand_landmarks = np.array(hand_landmarks)[0]
    else:
        hand_landmarks = np.zeros(3)

    return torch.from_numpy(hand_landmarks.astype('float32'))


In [15]:
# Define model
GESTURE_MAP = {'down': 0, 'left': 1, 'right': 2, 'up': 3}
def model(batch):
    y = torch.zeros(len(batch))
    for i, video in enumerate(batch):
        prev_s = None
        avg_ds = torch.zeros(8, 3)
        for s in video:
            if prev_s != None:
                ds = s - prev_s
                avg_ds += ds
            prev_s = s
        avg_ds = avg_ds / len(video)
        dx, dy, dz = avg_ds.sum((0))
        # print(dx, dy, dz)

        if abs(dx) > abs(dy):
            if dx > 0:   y[i] = GESTURE_MAP["right"]
            elif dx < 0: y[i] = GESTURE_MAP["left"]
        if abs(dx) < abs(dy):
            if dy > 0:   y[i] = GESTURE_MAP["up"]
            elif dy < 0: y[i] = GESTURE_MAP["down"]
        if abs(dx)<1e-1 and (dy)<1e-1:
            y[i]=5
        print(dx,dy)
    return torch.tensor(y)

In [17]:
# Define training loop
def test(model, data, label=""):
    correct = 0
    for k, (x,y) in enumerate(data):
        hand_landmarks = torch.zeros(batch_size, 8, 3)

        for i, video in enumerate(x):    # x is batch
            for j, frame in enumerate(video):
                hand_landmarks[i][j] = get_hand_landmarks_frame(frame, True)
        
        hand_landmarks = hand_landmarks
        y_hat = model(hand_landmarks)
        correct += (y_hat==y).sum()
    print(label+"accuracy:", round(float(correct/(k+1)/batch_size), 3))

In [ ]:
test(model, test_loader, "Test")

In [ ]:
def predict(model, data, label=""):
    hand_landmarks = torch.zeros(1, 8, 3)
    
    for i, video in enumerate(data):    # x is batch
        for j, frame in enumerate(video):
            hand_landmarks[i][j] = get_hand_landmarks_frame(frame, False)
    
    y_hat = model(hand_landmarks)
    print(GESTURE_MAP_NUM2STR[y_hat[0].item()])

def LiveVideoTest():
    # Captures videos
    num_batches = 20
    batch_size = 1
    frames_per_video = 8
    frame_rate = 100

    for n in range(num_batches):
        batch = []
        for v in range(batch_size):
            video = []
            for f in range(frames_per_video):
                ret, frame = cam.read()
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                video.append(frame)
                time.sleep(1/frame_rate)
                
            batch.append(video)
        predict(model,np.array(batch))
    cam.release()
    cv2.destroyAllWindows()

cam = cv2.VideoCapture(0)
try:
    LiveVideoTest()
except Exception as e:
    print("Error:", e)
    cam.release()
    cv2.destroyAllWindows()

In [31]:
cam = cv2.VideoCapture(0)
LiveVideoTest()

tensor(0.) tensor(0.)


C:\Users\PMLS\AppData\Local\Temp\ipykernel_9484\2796939799.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(y)


KeyError: 5.0

In [33]:
cam.release()
cv2.destroyAllWindows()